# installing packages

In [ ]:
!pip install PyPDF2
!pip install langchain_community
!pip install langchain
!pip install langchain_google_genai
!pip install langchainhub
!pip install faiss-cpu
!pip install -qU langchain-openai

In [ ]:
!pip install python-dotenv

In [ ]:
!pip install PyPDF2
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 3.4 MB/s eta 0:00:00


In [ ]:
!pip install pinecone-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.2/216.2 kB 3.8 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade --quiet  \
    langchain-pinecone \
    langchain-openai \
    langchain \
    langchain-community \
    pinecone-notebooks

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.9/215.9 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.5/327.5 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 986.7 kB/s eta 0:00:00


In [ ]:
from langchain_openai import OpenAIEmbeddings
import bs4
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAI
from dotenv import load_dotenv
import os
import markdown
import requests

# data

Dummy data of room finding company
https://drive.google.com/file/d/1ZtZ-Z4rjdHVvhzOTJWzcurnHwV1B9VxN/view?usp=sharing

# simple RAG

In [ ]:
import os
from langchain_pinecone import PineconeVectorStore

from pinecone import Pinecone, ServerlessSpec

import time

In [ ]:
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
import markdown
import random
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import google.generativeai as genai
from langchain_community.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
# from langchain_groq import ChatGroq
from google.colab import userdata

In [ ]:
import os
import time
from dotenv import load_dotenv
from PyPDF2 import PdfReader
from langchain import OpenAI, PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.embeddings import OpenAIEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from langchain_pinecone import PineconeVectorStore

from pinecone import Pinecone, ServerlessSpec
from google.colab import userdata
import time
import markdown
class ChatbotLogic:
    def __init__(self):
        load_dotenv()
        self.openai_api_key = userdata.get('OPENAI_API_KEY')
        self.pinecone_api_key = userdata.get('PINECONE_API_KEY')
        self.index_name = "testingpinecone"

        os.environ["OPENAI_API_KEY"] = self.openai_api_key
        os.environ["PINECONE_API_KEY"] = self.pinecone_api_key

        self.docsearch = None

    def initialize(self, pdf_docs):
        print("Initializing document search...")
        raw_text = self.get_pdf_text(pdf_docs)
        text_chunks = self.get_text_chunks(raw_text)
        self.docsearch = self.get_vector_store(text_chunks)
        print("Document search initialized successfully.")

    def get_pdf_text(self, pdf_docs):
        print("Extracting text from PDFs...")
        text = ""
        for pdf in pdf_docs:
            pdf_reader = PdfReader(pdf)
            for page in pdf_reader.pages:
                text += page.extract_text()
        print("Text extraction complete.")
        return text

    def get_text_chunks(self, text):
        print("Splitting text into chunks...")
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=300)
        text_chunks = text_splitter.split_text(text)
        print(f"Text split into {len(text_chunks)} chunks.")
        return text_chunks

    def get_vector_store(self, text_chunks):
        print("Setting up vector store...")
        embeddings = OpenAIEmbeddings()
        pc = Pinecone(api_key=self.pinecone_api_key)

        if self.index_name not in pc.list_indexes():
            print(f"Creating index '{self.index_name}'...")
            try:
                pc.create_index(
                    name=self.index_name,
                    dimension=1536,
                    metric="cosine",
                    spec=ServerlessSpec(cloud="aws", region="us-east-1"),
                )
                while not pc.describe_index(self.index_name).status["ready"]:
                    time.sleep(1)
                print("Index creation complete.")
            except:
                print("Index creation failed.")

        index = pc.Index(self.index_name)
        print(index.describe_index_stats())
        docsearch = PineconeVectorStore.from_texts(text_chunks, embeddings, index_name=self.index_name)
        print("Vector store setup complete.")
        print(docsearch)
        return docsearch

    def get_conversational_chain(self):
        print("Initializing conversational chain...")
        prompt_template = """
        Act as a chatbot (virtual assistance) and your name is Tom.
        Understand what the user wants and answer their questions based on the Context in a helpful way.
        Keep the conversation flowing and feel natural, like talking to a friend.
        If you're not sure, just say that you don't know the answer, don't try to make up an answer.\n\n

        Note if the Question is Greeting then Answer like this.
        Greeting: Hello
        Answer: Hello! How can I help you today?

        Greeting: Hi
        Answer: Hi! What would you like to chat about?

        Greeting: Hey there
        Answer: Hey there! What's on your mind?

        Greeting: Good morning/afternoon/evening
        Answer: Good [morning/afternoon/evening]! What can I do for you?

        Greeting: How are you?
        Answer: I'm doing well, thank you for asking! How about you?

        Greeting: What's up? (informal)
        Answer: Not much, just hanging out in the digital world. What can I help you with?

        If the Question is not a Greeting then you can avoid a greeting altogether and jump right into addressing the Question.

        Context:\n {context}?\n
        Question: \n{question}\n
        """

        model = OpenAI(api_key=self.openai_api_key, model_name="gpt-3.5-turbo-instruct")
        prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
        chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)
        print("Conversational chain initialized.")
        return chain

    def user_input(self, user_question):
        print("Handling user input...")
        if not self.docsearch:
            raise ValueError("Document search index not initialized. Call initialize() first.")

        docs = self.docsearch.similarity_search(user_question)
        print(f"Found {len(docs)} documents related to the user's question.")
        print("docs ", docs)

        chain = self.get_conversational_chain()
        response = chain({"input_documents": docs, "question": user_question}, return_only_outputs=True)
        markdown_text = response['output_text']

        # Render the markdown using a dedicated library like `markdown`
        html_content = markdown.markdown(markdown_text)
        print("User input handled successfully.")
        return html_content

## initialization

In [ ]:
# Initialize RAG assistance
chatbot = ChatbotLogic()

# Set the PDFs you want to process
pdf_docs = ['/content/room-finder-company-info.pdf']

# Initialize the chatbot with the PDF documents
chatbot.initialize(pdf_docs)

Initializing document search...
Extracting text from PDFs...
Text extraction complete.
Splitting text into chunks...
Text split into 12 chunks.
Setting up vector store...
Creating index 'testingpinecone'...
Index creation complete.
{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}
Vector store setup complete.
Document search initialized successfully.


## Conversation

In [ ]:
response = chatbot.user_input('hi')
print(response)

Handling user input...
Found 4 documents related to the user's question.
docs  [Document(page_content='• Career Opportunities: Visit our Careers page for current job openings and\ninternship opportunities\n• Partnerships: W e collaborate with universities, corporations, and reloca-\ntion services for preferred rates\n• Legal Resources: W e provide access to tenant rights information and can\nrefer users to local legal aid organizations\n• Financial Services: Rent reporting to credit bureaus to help build credit\nhistory\n• Subletting: W e offer a secure platform for tenants to list and manage\nshort-term sublets with landlord approval\n• Property Management T ools: Landlords can access our suite of manage-\nment tools, including tenant screening and rent collection\nInnovative F eatures\n1. AI-Powered Recommendations: Our algorithm learns your preferences to\nsuggest ideal properties\n42. Augmented Reality F urniture Placement: Visualize your furniture in po-\ntential rentals before mo

In [ ]:
response = chatbot.user_input('I am a student and I am looking for house.')
print(response)

Handling user input...
Found 4 documents related to the user's question.
docs  [Document(page_content='2. Proof of income (recent pay stubs, bank statements, or tax returns)\n3. Employment verification letter\n4. Rental history or references\n5. Credit report (we can run this for you with your permission)\n6. Application fee (varies by property)\n7. Proof of renter’s insurance (can be purchased through our platform)\n8. Cosigner information (if applicable)\n9. Student ID or enrollment verification (for student housing)\n10. Social Security Number or Individual T axpayer Identification Number (for\nbackground check)\nProcess to Rent a Room/Apartment\n1. Browse listings on our website or mobile app\n2. Use filters to narrow down your search\n3. Schedule virtual or in-person tours\n4. Submit an application for your chosen property\n5. Provide required documents\n6. Undergo a background and credit check\n7. Sign the lease agreement (digital signatures accepted)\n8. Pay the security deposit

In [ ]:
response = chatbot.user_input('what is the price of house for the students?')
print(response)

Handling user input...
Found 4 documents related to the user's question.
docs  [Document(page_content='1. Studio Apartments\n• Size: 300-500 sq ft\n• Amenities: Kitchenette, bathroom, living/sleeping area\n• Price Range: $800-$1500/month\n• Special F eatures: Some units offer murphy beds or convertible furni-\nture\n12. One-Bedroom Apartments\n• Size: 550-750 sq ft\n• Amenities: Separate bedroom, full kitchen, living room, bathroom\n• Price Range: $1000-$2000/month\n• Special F eatures: Select units have balconies or in-unit laundry\n3. Shared Rooms\n• Size: 150-250 sq ft per person\n• Amenities: Shared kitchen, bathroom, and common areas\n• Price Range: $400-$800/month per person\n• Special F eatures: Co-living spaces with organized community events\n4. Luxury Penthouses\n• Size: 1500+ sq ft\n• Amenities: Multiple bedrooms, high-end appliances, panoramic\nviews\n• Price Range: $5000+/month\n• Special F eatures: Concierge service, private elevators, roof terraces\n5. Student Housing\n•

# RAG with conversation memory and history

In [ ]:
import bs4
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
# from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter


class ChatbotLogic:
    def __init__(self):
        load_dotenv()
        self.openai_api_key = userdata.get('OPENAI_API_KEY')
        self.pinecone_api_key = userdata.get('PINECONE_API_KEY')
        self.index_name = "testingpinecone"

        os.environ["OPENAI_API_KEY"] = self.openai_api_key
        os.environ["PINECONE_API_KEY"] = self.pinecone_api_key

        self.llm = OpenAI(api_key=self.openai_api_key, model_name="gpt-3.5-turbo-instruct")

        print("Loading and processing PDF...")
        docs = ""
        for pdf in ['/content/room-finder-company-info.pdf']:
            pdf_reader = PdfReader(pdf)
            for page in pdf_reader.pages:
                docs += page.extract_text()

        print("Initializing embeddings and text splitting...")
        embeddings = OpenAIEmbeddings()
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        splits = text_splitter.split_text(docs)

        print("Setting up Pinecone index...")
        pc = Pinecone(api_key=self.pinecone_api_key)

        if self.index_name not in pc.list_indexes():
            print(f"Creating index '{self.index_name}'...")
            try:
                pc.create_index(
                    name=self.index_name,
                    dimension=1536,
                    metric="cosine",
                    spec=ServerlessSpec(cloud="aws", region="us-east-1"),
                )
                while not pc.describe_index(self.index_name).status["ready"]:
                    time.sleep(1)
                print("Index creation complete.")
            except Exception as e:
                print(f"Index creation failed: {e}")

        index = pc.Index(self.index_name)
        print(index.describe_index_stats())

        print("Setting up Pinecone vector store...")
        docsearch = PineconeVectorStore.from_texts(splits, embeddings, index_name=self.index_name)
        print("Vector store setup complete.")
        print(docsearch)
        retriever = docsearch.as_retriever()

        print("Setting up question-answering chains...")
        self.contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
        self.contextualize_q_prompt = ChatPromptTemplate.from_messages(
            [
                ("system", self.contextualize_q_system_prompt),
                MessagesPlaceholder("chat_history"),
                ("human", "{input}"),
            ]
        )
        self.history_aware_retriever = create_history_aware_retriever(
            self.llm, retriever, self.contextualize_q_prompt
        )

        self.qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

{context}"""
        self.qa_prompt = ChatPromptTemplate.from_messages(
            [
                ("system", self.qa_system_prompt),
                MessagesPlaceholder("chat_history"),
                ("human", "{input}"),
            ]
        )
        self.question_answer_chain = create_stuff_documents_chain(self.llm, self.qa_prompt)

        self.rag_chain = create_retrieval_chain(
            self.history_aware_retriever, self.question_answer_chain
        )

        self.store = {}

    def get_session_history(self, session_id: str) -> BaseChatMessageHistory:
        if session_id not in self.store:
            self.store[session_id] = ChatMessageHistory()
        return self.store[session_id]

    def process_message(self, session_id: str, user_input: str) -> str:
        conversational_rag_chain = RunnableWithMessageHistory(
            self.rag_chain,
            self.get_session_history,
            input_messages_key="input",
            history_messages_key="chat_history",
            output_messages_key="answer",
        )
        response = conversational_rag_chain.invoke(
            {"input": user_input},
            config={"configurable": {"session_id": session_id}},
        )
        return response

In [ ]:
# Create an instance of ChatbotLogic
assistant = ChatbotLogic()

# Define a session ID for the conversation (can be any unique identifier)
session_id = "123"

# User input
user_input = "Hi"

# Process the user input and get the response
response = assistant.process_message(session_id, user_input)

# Print the response
print("Assistant:", response["answer"])

Loading and processing PDF...
Initializing embeddings and text splitting...
Setting up Pinecone index...
Creating index 'testingpinecone'...
Index creation failed: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'x-pinecone-api-version': '2024-04', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'X-Cloud-Trace-Context': '06b9e8b18d97c37267a96220c7d4114e', 'Date': 'Sat, 29 Jun 2024 14:12:21 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 23}},
 'total_vector_count': 23}
Setting up Pinecone vector store...
Vector store setup complete.
Setting up ques

## conversation

In [ ]:
# User input
user_input = "I am looking for a vacation rooms"

# Process the user input and get the response
response = assistant.process_message(session_id, user_input)

# Print the response
print("Assistant:", response["answer"])

Assistant: 
AI: RoomEase Finder can help you find short-term vacation rooms as well as long-term rentals. Our platform has a vast network of property owners in major cities in the United States, making it easier for you to find your ideal living space. You can also control IoT devices in compatible units directly through our mobile app for a smart home experience. 


In [ ]:
# User input
user_input = "what are the required documents for buying a house?"

# Process the user input and get the response
response = assistant.process_message(session_id, user_input)

# Print the response
print("Assistant:", response["answer"])

Assistant: 
AI: To rent through RoomEase Finder, you’ll need to provide a valid government-issued ID (driver’s license, passport, etc.), proof of income (recent pay stubs, bank statements, or tax returns), employment verification letter, rental history or references, credit report (we can run this for you with your permission), application fee (varies by property), proof of renter’s insurance (can be purchased through our platform), cosigner information (if applicable), student ID or enrollment verification (for student housing), and Social Security Number or Individual Taxpayer Identification Number (for background check).


In [ ]:
# User input
user_input = "how to contact you?"

# Process the user input and get the response
response = assistant.process_message(session_id, user_input)

# Print the response
print("Assistant:", response["answer"])

Assistant: 
AI: You can contact us through our website, email, or phone number. Our customer support team is available 24/7 to assist you with any inquiries or concerns. You can also reach out to us through our social media channels. We strive to provide prompt and helpful assistance to all our users.


In [ ]:
# User input
user_input = "what are their details?"

# Process the user input and get the response
response = assistant.process_message(session_id, user_input)

# Print the response
print("Assistant:", response["answer"])

Assistant: 
AI: Our website is www.roomeasefinder.com, our email is support@roomeasefinder.com, and our phone number is 1-800-ROOMEASE (1-800-766-6327). You can also find us on Facebook, Instagram, Twitter, and LinkedIn. Our head office is located at 123 Main Street, Suite 500 New York, NY 10001, and we have regional offices on the West Coast, Midwest, and Southeast.
